<a href="https://colab.research.google.com/github/StefanATudose/NLP-Hackathon-2023/blob/main/Hackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive/NitroNLP/Hackaton")
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Preprocesare

In [ ]:
seed = 19


import numpy as np
np.random.seed(seed)
np.random.RandomState(seed)

import random
random.seed(seed)

import torch
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)

import tensorflow as tf
tf.random.set_seed(seed)

import jax
jax.random.PRNGKey(seed)

Array([ 0, 19], dtype=uint32)

In [ ]:
df = pd.read_csv("./train_data.csv")
test_df = pd.read_csv("./test_data.csv")

train_df = df[:9000]
#val_df = df[9000:]

In [ ]:
train_df.count

<bound method DataFrame.count of                                                     Text   Final Labels     Id
0      @CorinaTomescu05 Încă nu ...dar am trecut prin...  non-offensive      0
1                       @emosaphicbitch sau rosu ca mine  non-offensive      1
2      @DanaMinodora Ce frumoasa ești.. Arăți foarte ...  non-offensive      2
3      Din fericire în extaz!Ai dus covoareleeee?? Pu...      offensive      3
4      cand aveam vreo 5 ani credeam ca romana e sing...  non-offensive      4
...                                                  ...            ...    ...
39003  @violetAndro @CorinaCg3 Așa e, știu atâtea exe...  non-offensive  39003
39004  Directorul medical SAJ Galați, despre dispecer...  non-offensive  39004
39005  Au "invatat" prostii de balta de la Satan ce e...  non-offensive  39005
39006  @_seriosul_ E o parte de adevar. Dar pana la u...  non-offensive  39006
39007  @libertatea Să avem răbdare, să vedem ce spune...      offensive  39007

[39008 rows x 3 co

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(train_df["Text"])

train_data = vectorizer.transform(train_df["Text"])
#val_data = vectorizer.transform(val_df["Text"])
test_data = vectorizer.transform(test_df["Text"])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
scaler.fit(train_data)

scaled_x_train = scaler.transform(train_data)
#scaled_x_val = scaler.transform(val_data)
scaled_x_test = scaler.transform(test_data)

In [ ]:
from sklearn import svm

def svm_classifier(train_data, train_labels, test_data, c):
    linear_svm_classifier = svm.SVC(C=c, kernel='sigmoid') # sigmoid
    linear_svm_classifier.fit(train_data, train_labels)
    predicted_labels_train = linear_svm_classifier.predict(train_data)
    predicted_labels_test = linear_svm_classifier.predict(test_data)

    return predicted_labels_train, predicted_labels_test

def compute_accuracy(gt_labels, predicted_labels):
    accuracy = np.sum(predicted_labels == gt_labels) / len(predicted_labels)
    return accuracy


In [ ]:
predicted_labels_train, predicted_labels_test = svm_classifier(scaled_x_train, train_df["Final Labels"], \
                                                               scaled_x_test, \
                                                               0.75)
#print("Accuracy: ", compute_accuracy(val_df["Final Labels"], predicted_labels_test))

In [ ]:
sub = pd.read_csv("./sample_submission.csv")

lista1 = list(range(len(predicted_labels_test)))

d = {'Id': lista1, 'Label': predicted_labels_test}
sub = pd.DataFrame(data=d)

sub.to_csv("./sub1.csv", index=False)
